In [1]:
import numpy as np
import pandas as pd
import pickle
import statsmodels.api as sm

In [2]:
data_dir = '/home/simon/Documents/Bodies/data/OD_dataframes_compiled/'

with open(f'{data_dir}bodies_df_fatal.pkl', 'rb') as file:
    bodies_df_fatal = pickle.load(file)

In [9]:
bodies_df_fatal

,person_mean,person_median,person_fasterR50,person_fasterR101,person_fasterX101,person_retinaR50,person_retinaR101,child_mean,child_median,child_retinaR50,...,longitude_full,latitude_full,year_month,month_id,new_year,index_right,gid,log_best,log_high,log_low
0,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,...,44.378799,33.302431,2008-12,348.0,2008,9143622.0,177569.0,1.791759,1.791759,1.791759
1,3.0,3.0,3.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,...,44.378799,33.302431,2009-09,357.0,2009,9143622.0,177569.0,1.791759,1.791759,1.791759
2,1.6,2.0,2.0,1.0,1.0,2.0,2.0,0.0,0.0,0.0,...,44.378799,33.302431,2007-10,334.0,2007,9143622.0,177569.0,3.367296,4.174387,3.367296
3,2.8,3.0,2.0,3.0,3.0,3.0,3.0,0.0,0.0,0.0,...,41.037860,34.427567,2005-11,311.0,2005,9334635.0,179003.0,4.941642,5.056246,4.941642
4,1.8,2.0,1.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,...,41.046437,34.390889,2005-11,311.0,2005,9334635.0,179003.0,4.941642,5.056246,4.941642
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152427,0.6,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,44.152789,34.001313,2004-01,289.0,2004,9336507.0,179009.0,0.000000,0.000000,0.000000
152428,2.0,2.0,2.0,2.0,2.0,2.0,2.0,0.0,0.0,0.0,...,44.152789,34.001313,2004-01,289.0,2004,9336507.0,179009.0,0.000000,0.000000,0.000000
152429,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,44.152789,34.001313,2004-01,289.0,2004,9336507.0,179009.0,0.000000,0.000000,0.000000
152430,4.8,5.0,5.0,5.0,5.0,5.0,4.0,1.5,1.5,2.0,...,44.152789,34.001313,2004-01,289.0,2004,9336507.0,179009.0,0.000000,0.000000,0.000000


In [3]:
list_of_features_mean = [feature for feature in bodies_df_fatal.columns if 'mean' in feature]


In [4]:
X = bodies_df_fatal[list_of_features_mean]
y = bodies_df_fatal['log_best']

X2 = sm.add_constant(X)
est = sm.OLS(y, X2)
est2 = est.fit()
print(est2.summary())

                            OLS Regression Results                            
Dep. Variable:               log_best   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     46.34
Date:                Fri, 17 Jun 2022   Prob (F-statistic):          4.63e-147
Time:                        01:01:57   Log-Likelihood:            -2.9476e+05
No. Observations:              152432   AIC:                         5.895e+05
Df Residuals:                  152415   BIC:                         5.897e+05
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
const             